In [5]:
# Install dependencies (Colab)
!pip install --quiet pillow imageio

# %%
# Imports & enums
from enum import Enum
from dataclasses import dataclass
import heapq
import math
import random
import os
from PIL import Image, ImageDraw, ImageFont
import imageio
import numpy as np

# %%
# Enums
class Dir(Enum):
    UP = (0, -1)
    DOWN = (0, 1)
    LEFT = (-1, 0)
    RIGHT = (1, 0)

    @property
    def vec(self):
        return self.value

# %%
# Grid and Map utilities
class Grid:
    def __init__(self, width, height, walls=None, food=None):
        self.w = width
        self.h = height
        self.walls = set(walls) if walls else set()
        self.food = set(food) if food else set()

    def in_bounds(self, pos):
        x, y = pos
        return 0 <= x < self.w and 0 <= y < self.h

    def passable(self, pos):
        return pos not in self.walls

    def neighbors(self, pos):
        (x, y) = pos
        results = [(x+1,y),(x-1,y),(x,y+1),(x,y-1)]
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return list(results)

# %%
# A* Implementation

def heuristic(a, b):
    # Manhattan distance for grid
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def a_star_search(grid: Grid, start, goal, ghost_positions=None, predicted_ghosts=None):
    """A* with ghost avoidance + predicted ghost future tiles."""
    ghost_positions = set(ghost_positions or [])
    predicted_ghosts = set(predicted_ghosts or [])

    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            break

        for nxt in grid.neighbors(current):
            # avoid ghosts and predicted ghost tiles
            if nxt in ghost_positions or nxt in predicted_ghosts:
                continue

            new_cost = cost_so_far[current] + 1

            # penalize adjacency to danger zones
            for g in ghost_positions.union(predicted_ghosts):
                if abs(nxt[0] - g[0]) + abs(nxt[1] - g[1]) == 1:
                    new_cost += 6

            if nxt not in cost_so_far or new_cost < cost_so_far[nxt]:
                cost_so_far[nxt] = new_cost
                priority = new_cost + heuristic(goal, nxt)
                heapq.heappush(frontier, (priority, nxt))
                came_from[nxt] = current

    if goal not in came_from:
        return []

    path = []
    cur = goal
    while cur != start:
        path.append(cur)
        cur = came_from[cur]
    path.reverse()

    return path

# %%
# Agent & Ghost classes
@dataclass
class Pacman:
    pos: tuple

    def step(self, next_pos):
        self.pos = next_pos

@dataclass
class Ghost:
    pos: tuple
    rng_walk: bool = True

    def step(self, grid: Grid, pac_pos):
        # Simple ghost logic: random walk but biased towards Pacman
        choices = grid.neighbors(self.pos)
        if not choices:
            return
        if random.random() < 0.6:
            # move towards pacman with probability
            choices.sort(key=lambda p: heuristic(p, pac_pos))
            self.pos = choices[0]
        else:
            self.pos = random.choice(choices)

# %%
# Rendering utilities (PIL)

def render_grid(grid: Grid, pac: Pacman, ghosts: list, cell_size=32, show_grid=True, path=None):
    W, H = grid.w * cell_size, grid.h * cell_size
    img = Image.new('RGB', (W, H), 'black')
    draw = ImageDraw.Draw(img)

    # draw walls
    for (x,y) in grid.walls:
        draw.rectangle([x*cell_size, y*cell_size, (x+1)*cell_size-1, (y+1)*cell_size-1], fill=(50,50,120))

    # draw food
    for (x,y) in grid.food:
        cx = x*cell_size + cell_size//2
        cy = y*cell_size + cell_size//2
        r = max(2, cell_size//8)
        draw.ellipse([cx-r, cy-r, cx+r, cy+r], fill=(255,255,0))

    # optional draw planned path
    if path:
        for (x,y) in path:
            draw.rectangle([x*cell_size+cell_size//4, y*cell_size+cell_size//4,
                            (x+1)*cell_size-cell_size//4, (y+1)*cell_size-cell_size//4], outline=(0,255,0))

    # draw pacman
    px, py = pac.pos
    draw.ellipse([px*cell_size+2, py*cell_size+2, (px+1)*cell_size-2, (py+1)*cell_size-2], fill=(255,200,0))

    # draw ghosts
    for g in ghosts:
        gx, gy = g.pos
        draw.ellipse([gx*cell_size+4, gy*cell_size+4, (gx+1)*cell_size-4, (gy+1)*cell_size-4], fill=(255,0,0))

    return img

# %%
# Game loop & automated agent
class PacmanGame:
    def __init__(self, grid: Grid, pac: Pacman, ghosts: list, max_steps=2000):
        self.grid = grid
        self.pac = pac
        self.ghosts = ghosts
        self.max_steps = max_steps
        self.frames = []
        self.step_count = 0
        os.makedirs('frames', exist_ok=True)

    def nearest_food(self):
        if not self.grid.food:
            return None
        # choose nearest by manhattan
        return min(self.grid.food, key=lambda f: heuristic(self.pac.pos, f))

    def run(self, render_every=1, save_video=True):
        while self.grid.food and self.step_count < self.max_steps:
            target = self.nearest_food()
            # compute ghost positions for avoidance
            ghost_positions = [g.pos for g in self.ghosts]
            # predicted ghost moves
            predicted = []
            for g in self.ghosts:
                neigh = self.grid.neighbors(g.pos)
                if neigh:
                    neigh.sort(key=lambda p: heuristic(p, self.pac.pos))
                    predicted.append(neigh[0])
            path = a_star_search(self.grid, self.pac.pos, target, ghost_positions, predicted_ghosts=predicted)
            # if no path, break to avoid infinite loop
            if not path:
                print('No path to target, stopping.')
                break
            # take one step along path
            next_pos = path[0]
            # apply pacman move
            self.pac.step(next_pos)
            # collect food
            if self.pac.pos in self.grid.food:
                self.grid.food.remove(self.pac.pos)

            # ghosts move
            for g in self.ghosts:
                g.step(self.grid, self.pac.pos)

            # check collisions
            for g in self.ghosts:
                if g.pos == self.pac.pos:
                    print(f'Pacman caught by ghost at step {self.step_count}!')
                    # end game as failure
                    self.step_count = self.max_steps
                    break

            # render
            if self.step_count % render_every == 0:
                img = render_grid(self.grid, self.pac, self.ghosts, cell_size=32, path=path)
                fn = f'frames/frame_{self.step_count:04d}.png'
                img.save(fn)
                self.frames.append(fn)

            self.step_count += 1

        # final render
        img = render_grid(self.grid, self.pac, self.ghosts, cell_size=32)
        fn = f'frames/frame_{self.step_count:04d}.png'
        img.save(fn)
        self.frames.append(fn)

        if save_video:
            images = [imageio.imread(f) for f in self.frames]
            imageio.mimsave('pacman_demo.mp4', images, fps=8)
            print('Saved pacman_demo.mp4 and frames in /content/frames')

# %%
# Small test grid (3x3) as recommended for beginners

def make_test_grid_3x3():
    w, h = 3, 3
    walls = {(1,1)}  # center blocked
    food = {(0,0), (2,2)}
    grid = Grid(w,h,walls=walls, food=food)
    pac = Pacman(pos=(0,1))
    ghosts = [Ghost(pos=(2,1))]
    return grid, pac, ghosts

# %%
# Full demo: larger map

def make_demo_grid():
    w, h = 15, 11
    walls = set()
    # create a perimeter wall
    for x in range(w):
        walls.add((x,0)); walls.add((x,h-1))
    for y in range(h):
        walls.add((0,y)); walls.add((w-1,y))
    # some interior walls
    for x in range(3,12):
        walls.add((x,4))
    for y in range(2,9):
        walls.add((6,y))
    # generate food everywhere not wall except starting positions
    food = set((x,y) for x in range(1,w-1) for y in range(1,h-1) if (x,y) not in walls)
    pac_start = (1,1)
    # remove some foods
    food.discard(pac_start)
    ghosts = [Ghost(pos=(13,9)), Ghost(pos=(11,2))]
    grid = Grid(w,h,walls=walls, food=food)
    pac = Pacman(pos=pac_start)
    return grid, pac, ghosts

# %%
# Run the test and demo
if __name__ == '__main__':
    # 1) run the 3x3 test
    grid, pac, ghosts = make_test_grid_3x3()
    game = PacmanGame(grid, pac, ghosts, max_steps=50)
    print('Running 3x3 test...')
    game.run(render_every=1, save_video=True)

    # move frames to a subfolder for clarity
    os.makedirs('frames_test_3x3', exist_ok=True)
    for f in game.frames:
        os.replace(f, os.path.join('frames_test_3x3', os.path.basename(f)))
    print('3x3 test complete. frames in frames_test_3x3 and video pacman_demo.mp4')

    # 2) run the larger demo
    grid, pac, ghosts = make_demo_grid()
    game2 = PacmanGame(grid, pac, ghosts, max_steps=4000)
    print('Running full demo (may take a while)...')
    game2.run(render_every=2, save_video=True)
    os.makedirs('frames_demo', exist_ok=True)
    for f in game2.frames:
        os.replace(f, os.path.join('frames_demo', os.path.basename(f)))
    print('Demo complete. frames in frames_demo and pacman_demo.mp4')


Running 3x3 test...
Saved pacman_demo.mp4 and frames in /content/frames
3x3 test complete. frames in frames_test_3x3 and video pacman_demo.mp4
Running full demo (may take a while)...
No path to target, stopping.
Saved pacman_demo.mp4 and frames in /content/frames
Demo complete. frames in frames_demo and pacman_demo.mp4


/tmp/ipython-input-2073816359.py:237: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f) for f in self.frames]


In [6]:
# %%
# Run the test and demo + auto-play final video in Colab
from IPython.display import HTML
from base64 import b64encode

def play_video(filename):
    """Auto-display mp4 video inside colab."""
    mp4 = open(filename, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width="640" height="480" controls autoplay loop>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

if __name__ == '__main__':
    # 1) run the 3x3 test
    grid, pac, ghosts = make_test_grid_3x3()
    game = PacmanGame(grid, pac, ghosts, max_steps=50)
    print('Running 3x3 test...')
    game.run(render_every=1, save_video=True)

    os.makedirs('frames_test_3x3', exist_ok=True)
    for f in game.frames:
        os.replace(f, os.path.join('frames_test_3x3', os.path.basename(f)))
    print('3x3 test complete.')

    # 2) run the larger demo
    grid, pac, ghosts = make_demo_grid()
    game2 = PacmanGame(grid, pac, ghosts, max_steps=4000)
    print('Running full demo...')
    game2.run(render_every=2, save_video=True)
    os.makedirs('frames_demo', exist_ok=True)
    for f in game2.frames:
        os.replace(f, os.path.join('frames_demo', os.path.basename(f)))
    print('Demo complete.')

    # 3) AUTO-PLAY final video output
    print("Playing final Pac-Man demo video:")
    display(play_video("pacman_demo.mp4"))


Running 3x3 test...
Saved pacman_demo.mp4 and frames in /content/frames
3x3 test complete.
Running full demo...
No path to target, stopping.


/tmp/ipython-input-2073816359.py:237: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(f) for f in self.frames]


Saved pacman_demo.mp4 and frames in /content/frames
Demo complete.
Playing final Pac-Man demo video:
